In [15]:
import numpy as np
import h5py
import scipy.io
np.random.seed(1337) # for reproducibility
import matplotlib.pyplot as plt

from keras.preprocessing import sequence
from keras.optimizers import RMSprop, Adam, SGD
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution1D, MaxPooling1D
from keras.regularizers import l2, activity_l1
from keras.constraints import maxnorm
from keras.layers.recurrent import LSTM, GRU
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras import initializations


In [22]:
print 'loading data'
validmat = scipy.io.loadmat('validALLcontinuous.mat')
testmat = scipy.io.loadmat('testALLcontinuous.mat')

trainmat = scipy.io.loadmat('trainALLcontinuous.mat')

loading data


In [23]:
X_train = np.transpose(np.array(trainmat['tr'][0][0][0]),axes=(0,2,1))
y_train = np.array(trainmat['tr'][0][0][1]).squeeze()

print X_train.shape
print y_train.shape


(23961, 4, 113)
(23961,)


In [27]:

print 'building model'
model = Sequential()
model.add(Convolution1D(32, 25, border_mode='same', input_shape=(4, 113)))
model.add(Activation('relu'))
model.add(Convolution1D(32, 25, border_mode='same'))
model.add(Activation('relu'))
model.add(Convolution1D(32, 25, border_mode='same'))
model.add(Activation('relu'))
model.add(Flatten())

model.add(Dense(input_dim=113, output_dim=1))
# model.add(Activation('softmax'))

building model


In [28]:
print 'compiling model'
lr = 1e-5#learning rate
reg = 1e-6
adam = Adam(lr=lr, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0005)

model.compile(loss='mse', optimizer='adam', metrics=["mse"])


compiling model


In [29]:
print 'running at most 60 epochs'

checkpointer = ModelCheckpoint(filepath="bestmodel.hdf5", verbose=1, save_best_only=True)
earlystopper = EarlyStopping(monitor='val_loss', patience=2, verbose=1)

model.fit(X_train, y_train, batch_size=128, nb_epoch=60,  validation_data=(np.transpose(validmat['v'][0][0][0],axes=(0,2,1)), validmat['v'][0][0][1]), callbacks=[checkpointer,earlystopper])


running at most 60 epochs
Train on 23961 samples, validate on 2995 samples
Epoch 1/60
23961/23961 [==============================] - 10s - loss: 0.9783 - mean_squared_error: 0.9783 - val_loss: 1.0207 - val_mean_squared_error: 1.0207
Epoch 2/60
23961/23961 [==============================] - 10s - loss: 0.9559 - mean_squared_error: 0.9559 - val_loss: 1.0167 - val_mean_squared_error: 1.0167
Epoch 3/60
23961/23961 [==============================] - 10s - loss: 0.9406 - mean_squared_error: 0.9406 - val_loss: 1.0165 - val_mean_squared_error: 1.0165
Epoch 4/60
23961/23961 [==============================] - 11s - loss: 0.9128 - mean_squared_error: 0.9128 - val_loss: 1.0386 - val_mean_squared_error: 1.0386
Epoch 5/60
23961/23961 [==============================] - 10s - loss: 0.8646 - mean_squared_error: 0.8646 - val_loss: 1.0545 - val_mean_squared_error: 1.0545
Epoch 6/60
23961/23961 [==============================] - 11s - loss: 0.7968 - mean_squared_error: 0.7968 - val_loss: 1.1084 - val_mean

In [8]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution1d_1 (Convolution1D)  (None, 4, 32)         90432       convolution1d_input_1[0][0]      
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 4, 32)         0           convolution1d_1[0][0]            
____________________________________________________________________________________________________
convolution1d_2 (Convolution1D)  (None, 4, 32)         25632       activation_1[0][0]               
____________________________________________________________________________________________________
activation_2 (Activation)        (None, 4, 32)         0           convolution1d_2[0][0]            
___________________________________________________________________________________________